	# Author: Alexander Staub
	## Last changed: 2025.02.13
	## Purpose: Getting the chartmetrics IDs for a list of songs


In [25]:
#installing packages
import time
import requests
import logging
import pandas as pd

In [46]:
#Setup the logging of the errors
logging.basicConfig(
    filename='chartmetric_api.log',
    level=logging.INFO,
    format='%(asctime)s %(levelname)s: %(message)s'
)

In [47]:
# Define API host and your refresh token
HOST = 'https://api.chartmetric.com'
with open("chartmetric_refresh_token.txt", "r") as f:
    REFRESH_TOKEN = f.read().strip()

In [48]:
# Retrieve an access token using the refresh token
token_response = requests.post(f'{HOST}/api/token', json={'refreshtoken': REFRESH_TOKEN})

# Check if the token was retrieved successfully
if token_response.status_code != 200:

    # Log the error and raise an exception
    logging.error(f"Token retrieval error: {token_response.status_code}")
    raise Exception(f"Error: received {token_response.status_code} from /api/token")

# Extract the access token from the response
access_token = token_response.json()['token']

# Define the headers for the API requests
headers = {'Authorization': f'Bearer {access_token}'}

# Defining the get_request

Robust request logic that:
- backs off for a max of 26 hours in retries
- logs all erros it encounters


In [30]:


# --- Robust get_request Function ---
def get_request(endpoint, params=None, max_retries=5):
    backoff = 1  # initial backoff in seconds (used if header data is missing)
    for attempt in range(max_retries):
        try:
            response = requests.get(f"{HOST}{endpoint}", headers=headers, params=params)
        except Exception as ex:
            logging.error(f"Network error on attempt {attempt+1} for {endpoint}: {ex}")
            time.sleep(backoff)
            backoff *= 2
            continue

# Log the response status code and rate limit headers
        logging.info(f"Request to {endpoint} returned {response.status_code}. RateLimit headers: {response.headers}")

# Check if the response status code is 200
        if response.status_code == 200:
            return response.json()

# Handle different types of errors
# 401: Token may have expired; refresh it
        elif response.status_code == 401:
            # Token may have expired; refresh it
            logging.warning(f"401 error for {endpoint}. Refreshing token.")
            token_response = requests.post(f'{HOST}/api/token', json={'refreshtoken': REFRESH_TOKEN})
            if token_response.status_code != 200:
                logging.error(f"Token refresh failed: {token_response.status_code}")
                raise Exception(f"Token refresh failed with status {token_response.status_code}")
            new_token = token_response.json()['token']
            headers['Authorization'] = f'Bearer {new_token}'
            time.sleep(backoff)
            backoff *= 2

# 429: Rate limit exceeded; wait and retry
        elif response.status_code == 429:
            # Rate limit exceeded.
            reset_timestamp = response.headers.get("X-RateLimit-Reset")
            if reset_timestamp:
                # Wait until the time provided by the API
                sleep_time = int(reset_timestamp) - int(time.time())
                if sleep_time < 0:
                    sleep_time = backoff
            else:
                # No wait time provided by the API; compute one that totals 26 hours over all retries.
                total_wait_limit = 26 * 3600  # total wait time in seconds (26 hours)
                # Sum exponential weights for remaining attempts: for i from current attempt to max_retries-1
                remaining_weights = sum(2 ** i for i in range(attempt, max_retries))
                # Use the weight for the current attempt to assign a fraction of the total wait.
                sleep_time = total_wait_limit * (2 ** attempt / remaining_weights)
            logging.warning(f"429 error for {endpoint}. Sleeping for {sleep_time} seconds (attempt {attempt+1}/{max_retries}).")
            time.sleep(sleep_time)
            backoff *= 2

# 500: Server error; wait and retry
        elif response.status_code >= 500:
            logging.warning(f"Server error {response.status_code} for {endpoint}. Retrying after {backoff} seconds.")
            time.sleep(backoff)
            backoff *= 2

        else:
            logging.error(f"Error {response.status_code} for {endpoint}: {response.text}")
            raise Exception(f"Error: received {response.status_code} from {endpoint}")

# If the loop completes without returning, raise an exception
    raise Exception(f"Max retries exceeded for endpoint {endpoint}")

# Use isrc to access song characteristics:
- load in a subset of around 5000 of the spotify songs with spotify IDs
- Loop over them to get the chartmetric IDs and any other further information accessible
- use chartmetric IDs to get the information of relevance
- setup code in a way that allows us to get information for our relevant songs

In [50]:
#read in the first 8000 lines from the file "AD_spotify_accoustic_char_250k.csv" in the data/raw_data/Spotify folder and call it spotify_sample
# the data directory is 2 directories down from the current directory
# it should be a pandas dataframe and the csv has headers

import pandas as pd

#after each time a set is loaded, offset the next set by the previous set's length
#spotify_sample = pd.read_csv("Z:/Data_alexander/data/raw_data/Spotify/AD_spotify_accoustic_char_250k.csv", nrows=1000)
offset = 29000
#skip by offset but keep first row as it is the header

spotify_sample = pd.read_csv("Z:/Data_alexander/data/raw_data/Spotify/AD_spotify_accoustic_char_250k.csv", skiprows=range(1, offset), nrows=29000)


In [51]:
# remove duplicate rows from the spotify_sample dataframe based on isrcs
spotify_sample = spotify_sample.drop_duplicates(subset='ISRC')


In [52]:
# --- Function to Retrieve Chartmetric ID for an ISRC ---
def get_chartmetric_ids(isrc):
    endpoint = f"/api/track/isrc/{isrc}/get-ids"
    try:
        response = get_request(endpoint)
    
    # Log the response status code and rate limit headers
    except Exception as e:
        logging.error(f"Failed to get Chartmetric ID for ISRC {isrc}: {e}")
        return None

    # Expecting response["obj"] to be a non-empty list
    if response.get("obj") and isinstance(response["obj"], list) and len(response["obj"]) > 0:

        # Extract the chartmetric_ids from the first element of the list
        cm_ids = response["obj"][0].get("chartmetric_ids", None)

        # Check if cm_ids is a non-empty list
        if cm_ids and isinstance(cm_ids, list) and len(cm_ids) > 0:
            try:
                return float(cm_ids[0])
            
            # Log conversion errors
            except Exception as conv_err:
                logging.error(f"Conversion error for ISRC {isrc}: {conv_err}")
                return None
    return None



In [53]:
# Ensure the 'chartmetric_ids' column exists; if not, create it and fill with None.
if "chartmetric_ids" not in spotify_sample.columns:
    spotify_sample["chartmetric_ids"] = None

# --- Main Processing with Checkpointing & Throttling ---
checkpoint_file = "Z:/Data_alexander/data/incidental/chartmetric/chartmetric_track_id_checkpoints/chartmetric_ids_checkpoint.csv"
checkpoint_interval = 100  # save every 100 processed rows

for idx, row in spotify_sample.iterrows():
    isrc = row['ISRC']
    print(f"Processing row {idx}: ISRC = {isrc}")

    # Skip if ISRC already processed (if you restart from a checkpoint)
    if pd.notnull(spotify_sample.at[idx, "chartmetric_ids"]):
        print(f"Row {idx} already processed. Skipping.")
        continue

    try:
        chartmetric_id = get_chartmetric_ids(isrc)
    except Exception as e:
        print(f"Error processing ISRC {isrc} at row {idx}: {e}")
        logging.error(f"Error processing ISRC {isrc} at row {idx}: {e}")
        chartmetric_id = None

    spotify_sample.at[idx, "chartmetric_ids"] = chartmetric_id
    print(f"Row {idx} processed: ISRC = {isrc} -> Chartmetric ID = {chartmetric_id}")
    logging.info(f"Processed row {idx}, ISRC {isrc}: Chartmetric ID = {chartmetric_id}")

    # Optional: sleep briefly between requests to smooth out request rate.
    time.sleep(0.1)  # adjust as needed

    # Save a checkpoint periodically
    if idx % checkpoint_interval == 0 and idx > 0:
        spotify_sample.to_csv(checkpoint_file, index=False)
        print(f"Checkpoint saved at row {idx}")
        logging.info(f"Checkpoint saved at row {idx}")

Processing row 0: ISRC = FRZ115400100
Row 0 processed: ISRC = FRZ115400100 -> Chartmetric ID = 12318694.0
Processing row 10: ISRC = TCAAX1156222
Row 10 processed: ISRC = TCAAX1156222 -> Chartmetric ID = 26526772.0
Processing row 11: ISRC = USACX0511186
Row 11 processed: ISRC = USACX0511186 -> Chartmetric ID = 15025582.0
Processing row 12: ISRC = USSM15800335
Row 12 processed: ISRC = USSM15800335 -> Chartmetric ID = 15801869.0
Processing row 22: ISRC = USAR10500755
Row 22 processed: ISRC = USAR10500755 -> Chartmetric ID = 15050001.0
Processing row 23: ISRC = USVG29009224
Row 23 processed: ISRC = USVG29009224 -> Chartmetric ID = 15989531.0
Processing row 24: ISRC = USY1R0508712
Row 24 processed: ISRC = USY1R0508712 -> Chartmetric ID = 47439044.0
Processing row 25: ISRC = US58L0520908
Row 25 processed: ISRC = US58L0520908 -> Chartmetric ID = 44478260.0
Processing row 26: ISRC = FRO060200300
Row 26 processed: ISRC = FRO060200300 -> Chartmetric ID = 12259614.0
Processing row 33: ISRC = USSM

In [54]:
#create a backup of the scraped data just in case
spotify_sample_backup = spotify_sample

#transform the object type into float 
spotify_sample["chartmetric_ids"] = pd.to_numeric(spotify_sample["chartmetric_ids"], errors='coerce')

#transform into integer
spotify_sample["chartmetric_ids"] = spotify_sample["chartmetric_ids"].astype("Int64")



In [55]:
#retain only the rows with unique chartmetric_ids
spotify_sample_unique = spotify_sample.drop_duplicates(subset="chartmetric_ids")

Saving the files needs to take into account the version that I have already saved in the past as I am not able to run the code remotely

In [56]:

import os

# Define the base file path
base_filepath = "Z:/Data_alexander/data/incidental/chartmetric/chartmetric_ids_spotify_sample.csv"


# Split into directory, filename and extension
dir_name, base_filename = os.path.split(base_filepath)
name, ext = os.path.splitext(base_filename)

# Start with the original filename
final_filepath = base_filepath
counter = 1

# If the file exists, iterate until you get a filename that doesn't exist
while os.path.exists(final_filepath):
    final_filepath = os.path.join(dir_name, f"{name}_{counter}{ext}")
    counter += 1

# Save the dataframe to the final_filepath
spotify_sample_unique.to_csv(final_filepath, index=False)
print(f"Saved file as: {final_filepath}")

# Log the final message in the log file
logging.info("Completed processing all ISRC codes.")

Saved file as: Z:/Data_alexander/data/incidental/chartmetric/chartmetric_ids_spotify_sample.csv
